In [1]:
!pip install -U "transformers==4.40.0" --upgrade -q
!pip install accelerate bitsandbytes -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 23.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 73.3 MB/s eta 0:00:00


In [ ]:
import transformers
import torch

In [2]:
model_id = "unsloth/llama-3-8b-Instruct-bnb-4bit"
pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={
        "torch_dtype": torch.float16,
        "quantization_config": {"load_in_4bit": True},
        "low_cpu_mem_usage": True,
    },
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
/usr/local/lib/python3.10/dist-packages/transformers/quantizers/auto.py:159: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.
  warnings.warn(warning_msg)


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/131 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/51.1k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/459 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
# dummy conversation text
text="""
Selamat siang, saya Andi dari PLN. Saya ingin menyampaikan keluhan terkait layanan internet. Halo, selamat siang, Pak. Saya dengan Siti. Selamat siang, Bu. Siti dari mana, Bu? Saya dari Jakarta, Pak. Dari Pasar Minggu. Oh, baik.
Ini saya mau memastikan, Bu. Saya sudah membuat keluhan. Saya juga sudah mengirim email dan WhatsApp. Ini terkait dengan internet di rumah saya yang sering putus-putus. Sudah hampir dua minggu, tapi kok belum ada perbaikan, ya.
Saya bingung, Bu. Tadi saya sudah cek ke petugas lapangan, katanya sudah dijadwalkan, tapi sampai sekarang belum ada teknisi yang datang. Saya khawatir ini terus berlanjut. Nomor laporan saya sudah saya kirimkan melalui WhatsApp,
tapi belum ada tindak lanjut. Baik, Pak. Ini saya catat dulu, ya. Untuk laporan seperti ini biasanya ada tim khusus yang menangani. Bapak sudah pernah menghubungi nomor 0811-123-4567, belum? Sebentar, Bu. Saya cek dulu, ya.
Kemarin saya menghubungi nomor 0811-876-5432. Oh, nomor itu untuk layanan pelanggan umum, Pak. Untuk masalah teknis, Bapak bisa menghubungi nomor yang saya sebutkan tadi. Iya, Bu. Saya sudah coba semua nomor yang diberikan,
tapi belum ada respons yang memuaskan. Saya sudah email, WhatsApp, dan telepon, tapi tetap saja belum ada teknisi yang datang. Baik, Pak. Saya mengerti. Mohon maaf atas ketidaknyamanannya. Bapak bisa coba menghubungi nomor
WhatsApp khusus teknis di 0811-123-4567. Mereka biasanya merespons lebih cepat untuk masalah seperti ini. Iya, Bu. Tapi kemarin saya sudah coba juga, belum ada jawaban. Saya bingung harus bagaimana lagi. Apakah ada cara lain
supaya bisa segera diperbaiki? Saya sarankan Bapak mencoba lagi menghubungi nomor tersebut atau bisa datang langsung ke kantor pelayanan terdekat untuk mendapatkan penjelasan lebih lanjut. Karena kami dari tim layanan pelanggan
hanya bisa menyampaikan keluhan ke tim teknis. Baik, Bu. Terima kasih atas informasinya. Saya akan coba lagi. Kalau masih belum ada perbaikan, mungkin saya akan datang langsung ke kantor. Iya, Pak. Mohon maaf atas
ketidaknyamanannya. Terima kasih sudah menghubungi kami. Selamat siang. Selamat siang, Bu. Terima kasih.
"""

In [4]:
messages = [
    {"role": "system", "content": "Identifikasi masalah serta solusinya dalam bentuk list:"},
    {"role": "user", "content": text},
]

prompt = pipeline.tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
)
terminators = [
    pipeline.tokenizer.eos_token_id,
    pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
]
outputs = pipeline(
    prompt,
    max_new_tokens=256,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9,
)
print(outputs[0]["generated_text"][len(prompt):])

Berikut adalah identifikasi masalah serta solusinya dalam bentuk list:

**Masalah:**

1. Layanan internet di rumah Andi sering putus-putus dan belum ada perbaikan setelah dua minggu.
2. Andi sudah mengirimkan keluhan melalui email, WhatsApp, dan telepon, tapi belum ada respons yang memuaskan.
3. Petugas lapangan telah dijadwalkan, tapi teknisi belum datang.
4. Andi khawatir bahwa masalah ini terus berlanjut.

**Solusi:**

1. Andi dapat menghubungi nomor WhatsApp khusus teknis di 0811-123-4567 untuk mendapatkan respons yang lebih cepat.
2. Jika belum ada jawaban, Andi dapat mencoba menghubungi nomor tersebut lagi atau datang langsung ke kantor pelayanan terdekat untuk mendapatkan penjelasan lebih lanjut.
3. Andi dapat menghubungi tim teknis secara langsung untuk mendapatkan perbaikan yang lebih cepat.
4. Jika masih belum ada perbaikan, Andi dapat datang langsung ke kantor untuk mendapatkan penjelas
